In [8]:
#Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import joblib
import os.path
from messaging.telegrambot import Bot
import telegram

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import Adam # PODE MUDAR
import keras.callbacks as callbacks

In [2]:
#Loading dataset

dataset = '100k/' #100k dataset path
#dataset = '10M/' #10M dataset path
#dataset = '20M/' #20M dataset path

dataset_path = '../../Datasets/movieLens/' + dataset; # Full dataset path

filenames = {'movie': 'movie.csv', 'rating': 'rating.csv'};

time_zero = time.time();
#df_movies = pd.read_csv(dataset_path+filenames['movie'], error_bad_lines=False);
#print "[Loading Time:",time.time()-time_zero,"s]",filenames['movie'],'Shape: ', df_movies.shape;

#print df_movies.head();
#print df_movies.tail();

global df_ratings;
df_ratings = pd.read_csv(dataset_path+filenames['rating']);
print "[Loading Time:",time.time()-time_zero,"s]",filenames['rating'],'Shape: ', df_ratings.shape;


[Loading Time: 0.158306121826 s] rating.csv Shape:  (100000, 4)


In [3]:
print df_ratings.info();
print df_ratings.head();
print df_ratings.tail();


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
userId       100000 non-null int64
movieId      100000 non-null int64
rating       100000 non-null int64
timestamp    100000 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB
None
   userId  movieId  rating  timestamp
0     196      242       3  881250949
1     186      302       3  891717742
2      22      377       1  878887116
3     244       51       2  880606923
4     166      346       1  886397596
       userId  movieId  rating  timestamp
99995     880      476       3  880175444
99996     716      204       5  879795543
99997     276     1090       1  874795795
99998      13      225       2  882399156
99999      12      203       3  879959583


In [4]:
global usersIds;
global moviesIds;

def InitializeIds():
    global usersIds;
    global moviesIds;


    usersIds = np.zeros(1);
    moviesIds = np.zeros(1);


    for count in xrange(len(df_ratings['rating'])):

        if df_ratings['userId'][count] in usersIds:
            pass;
        else:
            usersIds = np.append(usersIds, df_ratings['userId'][count]);   

        if df_ratings['movieId'][count] in moviesIds:
            pass;
        else:
            moviesIds = np.append(moviesIds, df_ratings['movieId'][count]);
            currentMovieId = df_ratings['movieId'][count];

In [5]:
#Counting Number of Unique Users and Movies
time_zero = time.time();

InitializeIds();            
            
print "Users Ids array:\t",usersIds.shape
print "Movies Ids array:\t", moviesIds.shape;
print "Elapsed time: ", time.time()-time_zero; # Aprox. 370 s

Users Ids array:	(944,)
Movies Ids array:	(1683,)
Elapsed time:  2.98928284645


In [6]:
# Auxiliary Functions
global R;
global results_path;
results_path = './Results/';

def InitializeUtilityMatrix():
    InitializeIds();
    return np.zeros((usersIds.shape[0], moviesIds.shape[0]));

def FillUtilityMatrix():
    global R;
    for register in xrange(len(df_ratings['userId'])):
        userId = df_ratings['userId'][register];
        movieId = df_ratings['movieId'][register];

        #Getting userIndex from usersIds
        #itemindex = numpy.where(array==item) --> Return an array with indexes found
        userIndex = np.where(usersIds == userId);
        if len(userIndex[0]) == 0: # If userIndex hasn't found any matching user id in usersIds array
            print "UserId", userId, "not found in usersIds array.";
            pass;
        else:
            try:
                if len(userIndex[0]) > 1:
                    print "UserId ", userId, "is double-counted in usersIds.";
                userIndex = userIndex[0][0]; #Get the first occurance of userId match
            except IndexError:
                print "Error with user index: ", userIndex[0] ;
                pass;


        #Getting movieIndex from moviesIds
        #itemindex = numpy.where(array==item) --> Return an array with indexes found
        movieIndex = np.where(moviesIds == movieId);
        if len(movieIndex[0]) == 0: # If userIndex hasn't found any matching user id in usersIds array
            print "MovieId", movieId, "not found in usersIds array.";
            pass;
        else:
            try:
                if len(movieIndex[0]) > 1:
                    print "MovieId ", movieId, "is double-counted in moviesIds.";
                movieIndex = movieIndex[0][0]; #Get the first occurance of movieId match
            except IndexError:
                print "Error with movie index: ", movieIndex[0] ;
                pass;            

        R[userIndex][movieIndex] = df_ratings['rating'][register];
        
    return R;

def saveVariable(filename, variable):
    
    compression_parameter = 9;
    
    filepath = results_path+"Variables/"+dataset+filename;
    print "[*] Saving variable to " + filepath + " file...";
    if os.path.isfile(filepath): #Check if file already exists
        print "[*] Removing existing file...";
        os.remove(filepath);
        print "[+] Existing file removed.";    

    #joblib.dump(value, filename, compress=0, protocol=None, cache_size=None)
    joblib.dump(variable, filepath, compress = compression_parameter);
    print "[+] File saved.";   
import sys

time_zero = time.time();

R = InitializeUtilityMatrix();
R = FillUtilityMatrix();

print "Utility Matrix shape: ", R.shape;

KILO = 1000.0
MEGA = 1000*KILO;
GIGA = 1000*MEGA
print "Utility Matrix Memory Usage: ", sys.getsizeof(R)/(KILO)

#Saving variable
filename = 'originalUtilityMatrix';
print "[*] Saving original utility variable..."
saveVariable(filename, R); # saveVariable(filename, variable):
print "[+] Variable saved."


print "Elapsed time: ", time.time() - time_zero;

Utility Matrix shape:  (944, 1683)
Utility Matrix Memory Usage:  12710.128
[*] Saving original utility variable...
[*] Saving variable to ./Results/Variables/100k/originalUtilityMatrix file...
[*] Removing existing file...
[+] Existing file removed.
[+] File saved.
[+] Variable saved.
Elapsed time:  7.12597799301


In [7]:
print R.shape
#entrada_treinamento = R[:600,:]
#entrada_validacao = R[600:,:]

from sklearn.model_selection import train_test_split

entrada_treinamento, entrada_validacao = train_test_split(R)

print entrada_treinamento.shape
print entrada_validacao.shape

(944, 1683)
(708, 1683)
(236, 1683)


# Criando o modelo

In [11]:
from sklearn.externals import joblib

num_neurons = 16

# Usuario - Linha = Register
# Filme - COlunea = Feature

# Camada 0 -> Entrada 
# Camada 1 -> Hidden (SELU)
# Camada 2 -> Dropout
# Camada 3 -> Hidden (SELU)
# Camada 4 -> Saida (tngh)

# Entrada --> Hidden(Wk) --> Dropout --> Hidden(WkT) --> Entrada'

# Inicializando 
model = Sequential()

# Encoder
model.add(Dense(units=num_neurons, input_dim=R.shape[1], kernel_initializer='uniform', trainable = True))
# Trainable serve para eu treinar só uma camada. Posso fazer treinamento layer wise com isso
model.add(Activation('selu'))

# Camada de dropout
#model.add(Dropout(0.25))


model.add(Dense(units=num_neurons, kernel_initializer='uniform'))
model.add(Activation('selu'))

# Decoder
model.add(Dense(units=R.shape[1], kernel_initializer='uniform'))
model.add(Activation('tanh'))

model.compile(loss='mse',optimizer='Adam',metrics=['accuracy'])

model.summary()
#1683 --> Encoder(tanh) --> 10 --> Decoder --> 1683
# 1683 --> 1ª  Encoder Camada(750) --> 2ª Camada ... -> classificador (n neuronios)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 16)                26944     
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_4 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1683)              28611     
_________________________________________________________________
activation_5 (Activation)    (None, 1683)              0         
Total params: 55,827
Trainable params: 55,827
Non-trainable params: 0
_________________________________________________________________


In [ ]:
earlyStopping = callbacks.EarlyStopping(monitor='val_loss',
                                        patience=10,
                                        mode='auto')

trn_desc = model.fit(entrada_treinamento, entrada_treinamento,
                    epochs=100,
                    batch_size=1,
                    callbacks=[earlyStopping],
                    verbose=True,
                    validation_data=(entrada_validacao,entrada_validacao))


In [33]:
trn_desc.history['val_acc']

[0.0042372881355932203,
 0.0084745762711864406,
 0.0084745762711864406,
 0.012711864406779662,
 0.021186440677966101,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.0084745762711864406,
 0.021186440677966101,
 0.021186440677966101,
 0.021186440677966101,
 0.016949152542372881,
 0.016949152542372881,
 0.016949152542372881,
 0.021186440677966101,
 0.012711864406779662,
 0.016949152542372881,
 0.016949152542372881,
 0.012711864406779662,
 0.016949152542372881,
 0.012711864406779662,
 0.012711864406779662,
 0.016949152542372881,
 0.016949152542372881,
 0.016949152542372881,
 0.016949152542372881,
 0.016949152542372881,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.016949152542372881,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.012711864406779662,
 0.0127